# Chelsea Jaculina 011274913
DATA 266 Spring 2025 - HW 1

# Question 1 

In [7]:
## Install Requirements to Load Datasets
!pip install datasets transformers torch numpy pandas matplotlib nltk

Defaulting to user installation because normal site-packages is not writeable
  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
Using cached nltk-3.9.1-py3-none-any.whl (1.5 MB)
Using cached click-8.1.8-py3-none-any.whl (98 kB)


In [2]:
import torch
import time
import numpy as np
import pandas as pd
import tensorflow as tf
import openai
import nltk
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
from collections import Counter
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
nltk.download("punkt")

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/chelseajaculina/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## Load the IMDB dataset

In [3]:
from datasets import load_dataset 

# Load the IMDB dataset
imdb_ds = load_dataset("imdb") 

## Split the data into Training and Testing Samples

In [14]:
# 3000 training samples and 300 test samples
train_data = imdb_ds['train'].shuffle(seed = 42).select(range(3000))

test_data = imdb_ds['test'].shuffle(seed = 42).select(range(300))

## Zero-Shot Classification
Classifiying the test data without fine tuning.

In [9]:
from transformers import PreTrainedTokenizerFast

# Define tokenizers with vocabulary size 30,000
wordpiece_tokenizer = PreTrainedTokenizerFast.from_pretrained("distilbert-base-uncased")
bpe_tokenizer = PreTrainedTokenizerFast.from_pretrained("distilroberta-base")

# Set token configurations
wordpiece_tokenizer.add_special_tokens({"unk_token": "[UNK]", "pad_token": "[PAD]", 
                                        "cls_token": "[CLS]", "sep_token": "[SEP]", "mask_token": "[MASK]"})

bpe_tokenizer.add_special_tokens({"unk_token": "[UNK]", "pad_token": "[PAD]", 
                                  "cls_token": "[CLS]", "sep_token": "[SEP]", "mask_token": "[MASK]"})

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RobertaTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


5

In [15]:
def tokenize_function(examples, tokenizer):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

# Tokenize dataset using PreTrainedTokenizerFast
tokenized_train_wordpiece = train_data.map(lambda x: tokenize_function(x, wordpiece_tokenizer), batched=True)
tokenized_test_wordpiece = test_data.map(lambda x: tokenize_function(x, wordpiece_tokenizer), batched=True)

tokenized_train_bpe = train_data.map(lambda x: tokenize_function(x, bpe_tokenizer), batched=True)
tokenized_test_bpe = test_data.map(lambda x: tokenize_function(x, bpe_tokenizer), batched=True)

Map: 100%|██████████| 300/300 [00:00<00:00, 2234.29 examples/s]


In [24]:
from transformers import pipeline
from transformers import AutoModel

models = {
    "DistilBERT": "distilbert-base-uncased-finetuned-sst-2-english",
    "DistilRoBERTa": "distilroberta-base-finetuned-sst-2-english"
}

results = {}

for model_name, model_path in models.items():
    classifier = pipeline("sentiment-analysis", model=model_path, token="YOUR_HUGGINGFACE_TOKEN")
    test_texts = [item['text'] for item in test_data]  # Extract test samples
    
    # Process test_texts in smaller batches
    batch_size = 8
    labels = []
    for i in range(0, len(test_texts), batch_size):
        batch = test_texts[i:i+batch_size]
        predictions = classifier(batch, truncation=True, max_length=512)
        batch_labels = [1 if pred['label'] == 'POSITIVE' else 0 for pred in predictions]
        labels.extend(batch_labels)
    
    results[model_name] = labels

print("Classification completed successfully!")

Device set to use mps:0


OSError: There was a specific connection error when trying to load distilroberta-base-finetuned-sst-2-english:
401 Client Error: Unauthorized for url: https://huggingface.co/distilroberta-base-finetuned-sst-2-english/resolve/main/config.json (Request ID: Root=1-67c03498-6b48e9ac3120b6551f8df598;d2c66621-8b13-455a-af90-ac67f01552c6)

Invalid credentials in Authorization header

# Question 3